In [1]:
import torch
import gpytorch
import numpy as np
import pandas as pd
import scipy.optimize as optimize

from bo_functions import calc_y_exp
from bo_functions import create_y_data
from bo_functions import ExactGPModel
from bo_functions import train_GP_model
from bo_functions import calc_GP_outputs
from bo_functions import calc_ei_basic
from bo_functions import eval_GP_basic_tot
from bo_functions import create_sse_data
from bo_functions import eval_GP_basic_tot_scipy

from bo_plotters import plot_hyperparams
from bo_plotters import plot_xy
from bo_plotters import y_plotter
from bo_plotters import stdev_plotter
from bo_plotters import ei_plotter

In [2]:
#Pull x data from CSV
Theta_True = np.array([1,-1])
noise_std = 0.1**2
q = 2

exp_data_doc = "exp_data.csv"
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1]
Yexp = exp_data[:,2]
n = len(Xexp)
# print(x)

In [3]:
#Pull training data from CSV
train_data_doc = "train_2_in_data.csv"
train_data = np.array(pd.read_csv(train_data_doc, header=0,sep=","))
train_T = torch.tensor(train_data[:,1:3])
train_sse = torch.tensor(train_data[:,3])
# print(train_sse)

In [4]:
#Reduce Number of Training Points
t =len(train_T) #Desired number of training points
t = 3 #Set t=5 to show changing EI
train_T = train_T[0:t]
train_sse = train_sse[0:t]

In [5]:
#Define GP Testing space
#Define Testing Space
p=20
# Theta1 =  np.linspace(0.5,1.5,p) #1x10
# Theta2 =  np.linspace(-1.5,-0.5,p) #1x10
Theta1 =  np.linspace(-2,2,p) #1x10
Theta2 =  np.linspace(-2,2,p) #1x10
theta_mesh = np.array(np.meshgrid(Theta1, Theta2)) #2 Uniform 5x5 arrays
theta1_mesh = theta_mesh[0]
theta2_mesh = theta_mesh[1]
# print(theta_mesh)

In [6]:
#Training the GP Model
iterations = 300
explore_bias = 0
verbose = True

BO_iter = 50
train_T_dict = {}
train_sse_dict = {}
ei_dict = {}
sse_dict ={}
var_dict ={}
GP_mean_best_dict = {}
GP_var_best_dict = {}
GP_mean_min_dict ={}
GP_var_min_dict = {}
Theta_Opt_dict = {}
Theta_Best_dict = {}
Best_Error_dict = {}
if verbose == True:
    z_dict = {}
    ei_term_1_dict = {}
    ei_term_2_dict = {}
    CDF_dict = {}
    PDF_dict = {}

for i in range(BO_iter):
    print(i+1)
    if torch.is_tensor(train_T) != True:
        train_T = torch.from_numpy(train_T)
    if torch.is_tensor(train_sse) != True:
        train_sse = torch.from_numpy(train_sse)
    
    train_T_dict[i+1] = train_T
    train_sse_dict[i+1] = train_sse
    # initialize likelihood and model
    ##Assumes a homoskedastic noise model p(y | f) = f + noise
    likelihood = gpytorch.likelihoods.GaussianLikelihood()

    # We will use the simplest form of GP model, exact inference
    #Defines our model in terms of the class parameters in bo_functions
    model = ExactGPModel(train_T, train_sse, likelihood)
    train_GP = train_GP_model(model, likelihood, train_T, train_sse, iterations, verbose = False)
    
    ##Set Hyperparameters to 1
    outputscale = torch.tensor([1])
    lengthscale = torch.tensor([1])
    noise = torch.tensor([0.1])

    model.likelihood.noise = noise
    model.covar_module.base_kernel.lengthscale =lengthscale
    model.covar_module.outputscale = outputscale

    
    model.eval()
    #Puts likelihood in evaluation mode
    likelihood.eval()

    #Same point keeps being selected, should I remove that point by force?
    ei_components = eval_GP_basic_tot(p,theta_mesh, train_sse, model, likelihood, explore_bias, verbose)
    ei = ei_components[0]
    sse = ei_components[1]
#     print(sse)
    var = ei_components[2]
    stdev = ei_components[3]
    best_error = ei_components[4]
    if verbose == True:
        z = ei_components[5]
        ei_term_1 = ei_components[6]
        ei_term_2 = ei_components[7]
        CDF = ei_components[8]
        PDF = ei_components[9]
    
    ei_dict[i+1] = ei
    sse_dict[i+1] = sse
    var_dict[i+1] = var
    z_dict[i+1]=z
    
    Best_Error_dict[i+1] = best_error
    ei_term_1_dict[i+1] = ei_term_1
    ei_term_2_dict[i+1] = ei_term_2
    CDF_dict[i+1] = CDF
    PDF_dict[i+1] = PDF

    argmin = np.array(np.where(np.isclose(sse, np.amin(sse),atol=np.amin(sse)*1e-6)==True))
    Theta_1_Opt = float(theta1_mesh[argmin[0],argmin[1]])
    Theta_2_Opt = float(theta2_mesh[argmin[0],argmin[1]])
    Theta_Opt_GP = np.array((Theta_1_Opt,Theta_2_Opt))

    
    #calculates best theta value
    argmax = np.array(np.where(np.isclose(ei, np.amax(ei),atol=np.amax(ei)*1e-6)==True))
#     print(argmax)
    if len(argmax[0]) != 1:
        argmax = np.array([[argmax[0,1]],[argmax[1,1]]])
    
    Theta_1_Best = float(theta1_mesh[argmax[0],argmax[1]])
    Theta_2_Best = float(theta2_mesh[argmax[0],argmax[1]])
    Theta_Best = np.array((Theta_1_Best,Theta_2_Best))    
    
    #Finds the index where sse is the smallest and finds which Theta combination corresponds to that value
    theta0_b = Theta_Best
    theta0_o = Theta_Opt_GP
#     theta0_b = theta0_o = [1.5,-1.5] #Causes only this point to be repeated

    ## specify bounds
    # first array: lower bounds
    # second array: upper bounds
#     bnds = ([np.min(Theta1), np.min(Theta2)], [np.max(Theta1), np.max(Theta2)])
    bnds = [[np.min(Theta1), np.min(Theta2)], [np.max(Theta1), np.max(Theta2)]]

    ## use least squares optimizer in scipy?? IS LEAST SQUARES APPROPRIATE?
    # argument 1: function that takes theta as input, returns residual
    # argument 2: initial guess for theta
    # optional arguments 'bounds': bounds for theta
    # optional arugment 'args': additional arguments to pass to residual function
    # optional argument 'method': select the numerical method
    #   if you want to consider bounds, choose 'trf'
    #   if you do not want to consider bounds, try either 'lm' or 'trf'
    ei_sse_choice ="ei"
#     Best_Solution = optimize.least_squares(eval_GP_basic_tot_scipy, theta0_b,bounds=bnds, method='trf',args=((train_sse, model, likelihood, explore_bias, ei_sse_choice)))
    Best_Solution = optimize.minimize(eval_GP_basic_tot_scipy, theta0_b,bounds=bnds, method='Nelder-Mead',args=((train_sse, model, likelihood, explore_bias, ei_sse_choice)))

    #     print(Solution)
    theta_b = Best_Solution.x
    print("Theta Best = ",theta_b)
    print("Argmax Theta Best = ",Theta_Best)
    
    ei_sse_choice2 = "sse"
#     Opt_Solution = optimize.least_squares(eval_GP_basic_tot_scipy, theta0_o,bounds=bnds, method='trf',args=((train_sse, model, likelihood, explore_bias, ei_sse_choice2)))
    Opt_Solution = optimize.minimize(eval_GP_basic_tot_scipy, theta0_o,bounds=bnds, method='Nelder-Mead',args=((train_sse, model, likelihood, explore_bias, ei_sse_choice2)))

    theta_o = Opt_Solution.x
    print("Theta Opt = ",theta_o)
    print("Argmin Theta_Opt_GP = ",Theta_Opt_GP)
    ##Using this method allows for small changes, but these points aren't reflected by the graphs (innacurate?)
    
    Theta_Opt_dict[i+1] = Theta_Opt_GP
    Theta_Best_dict[i+1] = Theta_Best

    GP_mean_min = sse[argmin[0],argmin[1]]
    GP_var_min = (stdev[argmin[0],argmin[1]])**2
    GP_mean_best = sse[argmax[0],argmax[1]]
    GP_var_best = (stdev[argmax[0],argmax[1]])**2
    
    GP_mean_best_dict[i+1] = GP_mean_best
    GP_var_best_dict[i+1] = GP_var_best
    GP_mean_min_dict[i+1] = GP_mean_min
    GP_var_min_dict[i+1] = GP_var_min
    
    sse_title = "SSE"
    ei_plotter(theta_mesh, ei, Theta_True, theta_o, theta_b,train_T,plot_train=True)
#     y_plotter(theta_mesh, sse, Theta_True, theta_o, theta_b, train_T,sse_title,plot_train=True)
#     ei_plotter(theta_mesh, ei, Theta_True, Theta_Opt_GP, Theta_Best,train_T,plot_train=True)
#     y_plotter(theta_mesh, sse, Theta_True, Theta_Opt_GP, Theta_Best, train_T,sse_title,plot_train=True)
    ##Append best values to training data 
    #Convert training data to numpy arrays to allow concatenation to work
    train_T = train_T.numpy() #(q x t)
    train_sse = train_sse.numpy() #(1 x t)

    #Call the expensive function and evaluate at Theta_Best
    sse_Best = create_sse_data(q,theta_b, Xexp, Yexp) #(1 x 1)
#     sse_Best = create_sse_data(q,Theta_Best, Xexp, Yexp) #(1 x 1)
 
    #Add Theta_Best to train_p and y_best to train_y
    train_T = np.concatenate((train_T, [theta_b]), axis=0) #(q x t)
#     train_T = np.concatenate((train_T, [Theta_Best]), axis=0) #(q x t)
    train_sse = np.concatenate((train_sse, [sse_Best]),axis=0) #(1 x t)
    


1


/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:130: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution


ValueError: failed in converting 4th argument `xl' of _slsqp.slsqp to C/Fortran array

In [ ]:
print(train_T_dict)
print(train_sse_dict)
print(Best_Error_dict)
print("\n",ei_dict)
#This is Expected

In [ ]:
print("\nMax EIs")
for i in range(BO_iter):
    j=i+1
    print(np.amax(ei_dict[j]))

print("\nMin SSEs")
for i in range(BO_iter):
    j=i+1
    print(np.amin(sse_dict[j]))

##These are expected

print("\nBest EI GP Mean/Var")
print(GP_mean_best_dict)
print(GP_var_best_dict)

#GP Mean and EI Best vs Opt is the same. What does this mean?

print("\nLowest SSE GP Mean/Var")
print(GP_mean_min_dict)
print(GP_var_min_dict)
#Expected based on results above

for i in range(BO_iter):
    j = i+1
    argmax = np.array(np.where(np.isclose(ei_dict[j], np.amax(ei_dict[j]),atol=np.amax(ei_dict[j])*1e-6)==True))
    
    f_Best_a = Best_Error_dict[j][argmax[0],argmax[1]][0]
    print("\n f_Best",f_Best_a)
    print("EI_max:",np.amax(ei_dict[j]))
    print("GP Mean/Var:",GP_mean_best_dict[j],GP_var_best_dict[j])
    
    print("EI:",calc_ei_basic(f_Best_a,-GP_mean_best_dict[j],GP_var_best_dict[j], explore_bias))
#Max EIs are expected


print("\nTheta @ Lowest SSE")
print(Theta_Opt_dict)

#Expected based on results above
print("\nTheta @ Highest EI")
print(Theta_Best_dict)


# print("z values")
# print(z_dict[1],"\n", z_dict[2],"\n",z_dict[3])


# print("Term 1 values")
# print(ei_term_1_dict[1],"\n", ei_term_1_dict[2],"\n",ei_term_1_dict[3])

# print("Term 2 values")
# print(ei_term_2_dict[1],"\n", ei_term_2_dict[2],"\n",ei_term_2_dict[3])

# print("CDF")
# print(CDF[1],"\n", CDF[2],"\n",CDF[3])

# print("PDF")
# print(PDF[1],"\n", PDF[2],"\n",PDF[3])

In [ ]:
q_y = 3
y_GP_input = np.zeros((n,3))
for j in range(n):
    y_GP_input[j] = np.array([Theta_Best[0],Theta_Best[1],Xexp[j]])

y_GP_Opt = create_y_data(q_y,y_GP_input)
print(Theta_Opt_GP)
print(y_GP_Opt)
print(Yexp)

title = "XY Comparison"
plot_xy(Xexp, Yexp, y_GP_Opt,Theta_True,title)